<a href="https://colab.research.google.com/github/tomonari-masada/course2023-stats1/blob/main/1d_gaussian_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 混合正規分布を使った1次元データのクラスタリング

## 人工データの設定


In [ ]:
n_samples = 1000
n_clusters = 2

## 設定に基づいた人工データの生成

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(
    n_samples=n_samples,
    centers=n_clusters,
    n_features=1,
    cluster_std=0.5, # 問題を解きやすくするため。
    random_state=0)

In [ ]:
X

In [ ]:
y

## 人工データの可視化

In [ ]:
import matplotlib.pyplot as plt

plt.hist(X.reshape(-1), bins=50, lw=0);

## responsibilityの初期化

In [ ]:
import numpy as np
from scipy.special import softmax

q = softmax(np.random.randn(X.shape[0], n_clusters), axis=-1)

In [ ]:
q

## Mステップ

* 授業資料の$\mu_k, \sigma_k^2, \theta_k$の更新式をそのまま実装している。

In [ ]:
def M_step(X, q):
  mu = (q * X).sum(0) / q.sum(0)
  sigma_squared = (q * (X - mu) ** 2).sum(0) / q.sum(0)
  theta = q.sum(0) / q.shape[0]
  return mu, sigma_squared, theta

In [ ]:
mu, sigma_squared, theta = M_step(X, q)

In [ ]:
mu

In [ ]:
np.sqrt(sigma_squared)

In [ ]:
theta

## Eステップ

* 授業資料の$q_{i,k}$の更新式をそのまま実装している。

In [ ]:
def E_step(mu, sigma_squared, theta):
  conditional_likelihood = (
      np.exp(- (X - mu) ** 2 / (2 * sigma_squared))
      / np.sqrt(2 * np.pi * sigma_squared)
  )
  q = theta * conditional_likelihood
  return q / q.sum(-1, keepdims=True)

In [ ]:
q = E_step(mu, sigma_squared, theta)

In [ ]:
q

## EMアルゴリズムの実行

In [ ]:
for _ in range(10000):
  mu, sigma_squared, theta = M_step(X, q)
  q = E_step(mu, sigma_squared, theta)

In [ ]:
q

## クラスタリングの評価

In [ ]:
(q.argmax(-1) == y).sum()